In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 20:56:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet('/home/ezzaldin/data/taxi_tripdata/all_data_part.parquet')
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = true)



In [4]:
df.createOrReplaceTempView('data_view')

In [17]:
agg_green_df = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', pickup_datetime) AS revenue_hour, 
    -- Revenue calculation 
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) AS number_records
FROM
    data_view
WHERE service_type = 'green'
GROUP BY
    1, 2
""")

In [6]:
agg_green_df.show()

+------------+-------------------+----------------------------+--------------+
|revenue_zone|       revenue_hour|revenue_monthly_total_amount|number_records|
+------------+-------------------+----------------------------+--------------+
|         152|2022-03-28 20:00:00|                        18.5|             1|
|          52|2022-03-15 13:00:00|                       67.18|             2|
|         255|2022-05-29 21:00:00|          59.269999999999996|             3|
|          41|2022-05-01 19:00:00|                       38.86|             3|
|          95|2022-05-13 20:00:00|                      157.07|            11|
|          75|2022-05-18 07:00:00|          353.21999999999997|            19|
|          24|2022-05-09 08:00:00|                       42.71|             2|
|         226|2022-03-06 18:00:00|                       84.17|             3|
|          24|2022-05-01 16:00:00|                       78.15|             3|
|          95|2022-03-26 03:00:00|                  

In [7]:
agg_green_df.repartition(20)\
            .write\
            .parquet('/home/ezzaldin/data/taxi_tripdata/agg_revenue_green_taxi.parquet',
                     mode = 'overwrite')

In [18]:
agg_yellow_df = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', pickup_datetime) AS revenue_hour, 
    -- Revenue calculation 
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) AS number_records
FROM
    data_view
WHERE service_type = 'yellow'
GROUP BY
    1, 2
""")

In [9]:
agg_yellow_df.repartition(20)\
             .write\
             .parquet('/home/ezzaldin/data/taxi_tripdata/agg_revenue_yellow_taxi.parquet',
                      mode = 'overwrite')

In [19]:
agg_green_df = agg_green_df.withColumnRenamed('revenue_monthly_total_amount', 'revenue_monthly_total_amount_green')\
                           .withColumnRenamed('number_records', 'number_records_green')
agg_yellow_df = agg_yellow_df.withColumnRenamed('revenue_monthly_total_amount', 'revenue_monthly_total_amount_yellow')\
                            .withColumnRenamed('number_records', 'number_records_yellow')

In [20]:
merged_df = agg_green_df.join(agg_yellow_df,
                              on = ['revenue_zone', 'revenue_hour'],
                              how = 'outer')

In [21]:
merged_df.show()

+------------+-------------------+----------------------------------+--------------------+-----------------------------------+---------------------+
|revenue_zone|       revenue_hour|revenue_monthly_total_amount_green|number_records_green|revenue_monthly_total_amount_yellow|number_records_yellow|
+------------+-------------------+----------------------------------+--------------------+-----------------------------------+---------------------+
|           1|2022-01-01 13:00:00|                              null|                null|                             347.81|                    5|
|           1|2022-01-01 18:00:00|                              null|                null|                              110.3|                    1|
|           1|2022-01-02 05:00:00|                              null|                null|                             299.25|                    4|
|           1|2022-01-02 11:00:00|                              null|                null|                

In [22]:
merged_df.write.parquet('/home/ezzaldin/data/taxi_tripdata/merged.parquet',
                        mode = 'overwrite')

In [25]:
zones_df = spark.read.parquet('/home/ezzaldin/data/zones/')
zones_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [27]:
final_res_df = merged_df.join(zones_df,
                              merged_df.revenue_zone == zones_df.LocationID)\
                        .select('revenue_zone',
                                'revenue_hour',
                                'revenue_monthly_total_amount_green',
                                'revenue_monthly_total_amount_yellow')
final_res_df.show()

+------------+-------------------+----------------------------------+-----------------------------------+
|revenue_zone|       revenue_hour|revenue_monthly_total_amount_green|revenue_monthly_total_amount_yellow|
+------------+-------------------+----------------------------------+-----------------------------------+
|           1|2022-01-01 13:00:00|                              null|                             347.81|
|           1|2022-01-01 18:00:00|                              null|                              110.3|
|           1|2022-01-02 05:00:00|                              null|                             299.25|
|           1|2022-01-02 11:00:00|                              null|                               76.3|
|           1|2022-01-02 12:00:00|                              null|                             126.96|
|           1|2022-01-02 14:00:00|                              null|                              257.8|
|           1|2022-01-05 15:00:00|            